In [118]:
import sys
import numpy as np
import os
import pandas as pd
import pickle
sys.path.append('/home/ak/Documents/Research/hsmm/hsmm/')
_features_path= '/home/ak/Documents/Data/features_models/' #directory to save data
_models_path = os.path.join(_features_path,'models')
_path=os.getenv('FINANCE_DATA')
_ftse = os.path.join(_path,'FTSE/')
import matplotlib.pyplot as plt
figures_path= ('/home/ak/Documents/Research/QFPaper/paper/figures/')
%matplotlib inline

In [119]:
os.listdir(_models_path)

['ZLc1_3_model_results_.pkl',
 'ZLc1_2012_clfs_.pkl',
 'ZLc1_4_clfs_.pkl',
 'ZLc1_3_clfs_.pkl',
 'ZLc1_2012_model_results_.pkl']

In [76]:
from hsmm_core.observation_models import *
from hsmm_core.hmm import *
from hsmm_core.hmm_utils import load_data
from hsmm_core.prediction_engines import *

In [120]:
##useful functions##
def listdirs(folder): #return only directories from a master folder
    return [d for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))]


    

In [88]:
symbols_list= os.listdir(_ftse)
for _no, _ in enumerate(symbols_list):
    ticker = symbols_list[_no]
    no_days =len(os.listdir(_ftse+symbols_list[_no]))


In [79]:
# for _no, _ in enumerate(symbols_list):
#     ticker = symbols_list[_no]
#     data_path=os.path.join(_ftse+ticker)
#     listdirs(data_path)
#     no_days=len(os.listdir(os.path.join(_path+ticker)))


In [100]:
test_=(_features_path+ ticker)
os.makedirs(test_)

In [101]:
os.listdir(test_)

[]

In [102]:
#Three Hidden States
n_hidden_states = 3

zero_pc_threshold = 1e-5
init_params = {
    "obs_model_params": {
                                'epsilon': zero_pc_threshold,
                                'obs_model_name': 'ExpUniGauss',
                                'init_params': {'epsilon': zero_pc_threshold},
                                'em_init_method': initialization_method.cluster

    },
    "hidden_model_params": {
                                'no_hidden_states': n_hidden_states,
                                # 'pi':startprob,
                                # 'tpm': transmat
                                'em_init_method': initialization_method.uniform
    },
    "update_tag": 'tpsml'
}



In [103]:
# Look out for the flag which_trading_hours. Possible values are: 
# TradingHours.pre_open_hours, TradingHours.only_mkt_hours, TradingHours.all_trading_day 
# with the obvious interpretation
data_dic, data_sampling_tag =load_data(ticker,
                                       no_of_days=3, 
                                       #file_name=['20120217.csv'], 
                                       load_columns=['TradedTime', 'Duration','TradedPrice', 'ReturnTradedPrice'], 
                                       which_trading_hours=TradingHours.only_mkt_hours)

NameError: name 'TradingHours' is not defined

In [ ]:
labelling_methods = [
    {'method': dl_choices.control_chart,
  'cc_quantiles': [40, 70],
      'rolling_window': 15,
    },
    # {
    #     'method': dl_choices.control_chart,
    #     'cc_quantiles': [20, 80],
    #     'rolling_window': 10,
    # },
#     {
#         'method': dl_choices.simple_all,
#         'rolling_window': 5,
#     },
#     {
#         'method': dl_choices.simple_ma,
#         'rolling_window': 5,
#     },
    {
        'method': dl_choices.simple_ep,
        'rolling_window': 20,
        'cmp_threshold': 0.05, # Use this value to set a threshold in comparison, i.e. it does not suffice 
                            # to have P_t < P_{t + window} but  P_t < ( 1- cmp_threshold) * P_{t + window} 
    },
    {
        'method': dl_choices.simple_ep,
        'rolling_window': 20,
    },
]

labellers_ = []

for labelling_method in labelling_methods:
    method = labelling_method['method']

    if method is dl_choices.control_chart:
        # look for qunatile values, if not use default 10/90
        labellers_.append(data_labelling_cc(**labelling_method))
    else:
        labellers_.append(data_labelling_simple(**labelling_method))

In [ ]:

hmm_calibration_engine = hmm_calibration(no_parallel_procs=None,
                                         init_params=init_params)

trained_hmms = hmm_calibration_engine.hmm_fit_func(ticker, data_dic, 'resample__', force_recalc=True)

print trained_hmms

# a_model = trained_hmms['20140122']
# a_model.map_metrics(data_dic['20140122'])
#



In [ ]:
prediction_params = {
    'labelling_methods':  labelling_methods,
    'feature_generation': feature_generation.one_model_to_one_fset,
}

prediction_eng = hs_prediction_engine(hmm_init_params=init_params,
                                      prediction_innit_params=prediction_params,
                                      no_parallel_procs=None)

In [ ]:
features, _ = prediction_eng.train_model_data(ticker, data_dic, 'resample__', force_hmm_recalc=False)

In [ ]:
M=5
T=len(_dates)

_ll_results= {'_likeli_': {_no: {} for _no in xrange(2, 6)}}

for i in xrange(0,T):
    print _dates[i]

In [ ]:
for idx, _no in enumerate(xrange(2,6)):
    print _no
    init_params["hidden_model_params"]["no_hidden_states"] =_no
    hmm_calibration_engine = hmm_calibration(no_parallel_procs=None,
                                             init_params=init_params)
    trained_hmms = hmm_calibration_engine.hmm_fit_func(ticker, data_dic, 
                                                       'resample__', force_recalc=True)
    for _date in range(0,T):
        print _dates[_date]
        filename, _ = os.path.splitext(_dates[_date])
        _no_items=len(trained_hmms[filename].__dict__['history_likelihood_'])
        _likel_=trained_hmms[filename].__dict__['history_likelihood_'][_no_items-1]
        print _likel_
        _ll_results['_likeli_'][_no][_date] =_likel_


In [ ]:
for _date in range(0,T):
    filename, _ = os.path.splitext(_dates[_date])
    _no_items=len(trained_hmms[filename].__dict__['history_likelihood_'])
    _likel_=trained_hmms[filename].__dict__['history_likelihood_'][_no_items-1]
    print _likel_

In [ ]:
for i in xrange(2,6):
    print i

In [ ]:
_ll_results['_likeli_'][3]

In [ ]:
_ll_results['_likeli_'][5]

In [ ]:
_ll_results['_likeli_'][4]

In [ ]:
_no_states=_ll_results['_likeli_'].keys()

In [ ]:
dictionary = _ll_results['_likeli_']
print(dictionary.keys())

In [ ]:
print(dictionary[2])

In [ ]:
fig, ax = plt.subplots()
fig.autofmt_xdate()
for k in dictionary:
    xs, ys = zip(*sorted(dictionary[k].items(), key=lambda x: x[0]))
    ax.plot([datetime.strptime(_dates[i].split('.')[0], '%Y%m%d') for i in xs],
            ys,marker='o', linestyle='--', label=k)
ax.legend()
plt.xlabel('#Day')
plt.ylabel('likelihood')
plt.show()

In [ ]:
for _date in range(0,4):
    filename, _ = os.path.splitext(_dates[_date])
    print filename

In [ ]:
k